In [1]:
import importlib

package_name = "datasets"

try:
    importlib.import_module(package_name)
    print(f"{package_name} is already installed.")
except ImportError:
    print(f"{package_name} is not installed. Installing it now...")
    !pip install {package_name}
    print(f"{package_name} has been successfully installed.")

datasets is already installed.


In [2]:
import importlib

package_name = "accelerate"

try:
    importlib.import_module(package_name)
    print(f"{package_name} is already installed.")
except ImportError:
    print(f"{package_name} is not installed. Installing it now...")
    !pip install {package_name}
    print(f"{package_name} has been successfully installed.")

accelerate is already installed.


In [3]:
import pandas as pd
import numpy as np
from scipy.special import softmax
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

from transformers import Trainer
from transformers import TrainingArguments
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizer, AutoTokenizer, DataCollatorWithPadding, BertForSequenceClassification

from datasets import load_metric
from datasets import load_dataset
from datasets import load_from_disk

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
from google.colab import drive
drive.mount("/content/drive")

%cd "/content/drive/MyDrive/6386"

# from colab.file_utils import load_required
# load_required(force_reinstall_pytorch=True)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/6386


In [5]:
import torch
torch.cuda.empty_cache()

In [6]:
import os
print(os.getcwd())

/content/drive/MyDrive/6386


In [7]:
# !pip install datasets

In [8]:
raw_datasets = load_from_disk("finetune_data")
# checkpoint = "bert-base-uncased"
# checkpoint = "bert-base-cased"
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(example):
    return tokenizer(example["text"], padding=True, truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer) # pad all the examples to the length of the longest element when we batch elements together — dynamic padding.

Map:   0%|          | 0/2 [00:00<?, ? examples/s]

In [9]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 8
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 2
    })
    test: Dataset({
        features: ['text', 'input_ids', 'attention_mask'],
        num_rows: 10
    })
})

In [10]:
tokenized_datasets['train']['text'][:10]

['Wholesale inflation eases to 2.26% in February from 3.1% in previous month',
 'DISH Break out watch over 38.10  ',
 "$AMZN's stock faces pressure due to concerns about the company's third-party seller practices.",
 'ecent breakouts in DNKN GMC SBX appear to be the start of a group move. Still forming big cup bases. JVA also on watch.',
 "Despite the hype, $AMC's stock faces challenges as theater attendance remains uncertain.",
 'The company said that sales in the three months to the end of March slid to EUR86 .4 m US$ 113.4 m from EUR91 .2 m last year .',
 '- I am pleased that Bjorn Wahlroos has accepted the nomination .',
 "At the end of the review period , Nordic Aluminium 's order book stood at EUR 8.77 mn compared to EUR 7.04 in 2005 ."]

In [11]:
metric = load_metric("glue", "mrpc")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-11-093e3287862a>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("glue", "mrpc")


In [12]:
!pip install transformers[torch]
!pip install accelerate -U

In [13]:
import accelerate
print(accelerate.__version__)

0.24.1


In [14]:
training_args = TrainingArguments(
    output_dir="test-trainer",
    evaluation_strategy="epoch",
    num_train_epochs=5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=256,
    # During the first 500 training steps, the learning rate gradually increases from 0 (or a small base rate) to the specified learning rate.
    # This gradual increase helps in stabilizing the training process and often leads to better performance, as it prevents the model from making too large updates too quickly.
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='logs',
    logging_steps=10,  # how frequently the training progress is logged
    save_strategy="epoch",  # Set save strategy to "epoch" to save both best and last models
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none",  # disable wandb
    # fp16=True,  # Enable mixed precision training
)
# model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
model = BertForSequenceClassification.from_pretrained(checkpoint, num_labels=2).to(device)

trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

You are using a model of type distilbert to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['embeddings.LayerNorm.bias', 'encoder.layer.10.attention.output.LayerNorm.weight', 'encoder.layer.4.output.LayerNorm.bias', 'encoder.layer.0.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.2.intermediate.dense.weight', 'encoder.layer.3.output.dense.bias', 'encoder.layer.7.output.LayerNorm.bias', 'encoder.layer.8.output.dense.weight', 'encoder.layer.7.attention.output.LayerNorm.weight', 'encoder.layer.10.attention.self.value.bias', 'encoder.layer.5.intermediate.dense.bias', 'encoder.layer.7.attention.self.value.bias', 'encoder.layer.5.output.dense.bias', 'encoder.layer.5.intermediate.dense.weight', 'enco

In [15]:
trainer.train()

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.922338,0.000000,0.000000
2,No log,0.912998,0.000000,0.000000
3,No log,0.894486,0.000000,0.000000
4,No log,0.867271,0.000000,0.000000
5,No log,0.831929,0.000000,0.000000


TrainOutput(global_step=5, training_loss=0.7609110832214355, metrics={'train_runtime': 79.0206, 'train_samples_per_second': 0.506, 'train_steps_per_second': 0.063, 'total_flos': 842777599200.0, 'train_loss': 0.7609110832214355, 'epoch': 5.0})

In [16]:
output_dir = "saved_model"
os.makedirs(output_dir, exist_ok=True)

# Save the best model and the last model
trainer.save_model(output_dir)
tokenizer.save_pretrained(output_dir)

('saved_model/tokenizer_config.json',
 'saved_model/special_tokens_map.json',
 'saved_model/vocab.txt',
 'saved_model/added_tokens.json',
 'saved_model/tokenizer.json')

In [17]:
predictions = trainer.predict(tokenized_datasets["test"])
# print(predictions.predictions.shape, predictions.label_ids.shape)

In [18]:
predictions

PredictionOutput(predictions=array([[ 0.1966474 , -0.28340507],
       [ 0.20582949, -0.3134265 ],
       [ 0.17484556, -0.25611234],
       [ 0.23832574, -0.2616041 ],
       [ 0.24474546, -0.32449868],
       [ 0.19845586, -0.2908669 ],
       [ 0.2139138 , -0.22153285],
       [ 0.19264206, -0.24343744],
       [ 0.26210976, -0.29346582],
       [ 0.22881413, -0.28806487]], dtype=float32), label_ids=None, metrics={'test_runtime': 0.0924, 'test_samples_per_second': 108.262, 'test_steps_per_second': 10.826})

In [19]:
probabilities = softmax(predictions.predictions, axis=1)
predicted_labels = np.argmax(probabilities, axis=1)

print("Probabilities:\n", probabilities)
print("Predicted Labels:\n", predicted_labels)

Probabilities:
 [[0.61776024 0.38223973]
 [0.62697375 0.3730262 ]
 [0.6061024  0.39389762]
 [0.62244284 0.37755716]
 [0.6385888  0.36141124]
 [0.6199469  0.3800531 ]
 [0.60717356 0.39282647]
 [0.6073244  0.39267552]
 [0.6354282  0.36457178]
 [0.62641764 0.3735823 ]]
Predicted Labels:
 [0 0 0 0 0 0 0 0 0 0]


In [20]:
# true_labels = tokenized_datasets["test"]["label"]

# accuracy = accuracy_score(true_labels, predicted_labels)
# precision = precision_score(true_labels, predicted_labels)
# recall = recall_score(true_labels, predicted_labels, average='weighted')
# f1 = f1_score(true_labels, predicted_labels, average='weighted')

# print("Accuracy:", accuracy)
# print("Precision:", precision)
# print("Recall:", recall)
# print("F1 Score:", f1)

In [21]:
# helper_model.print_wrong_classifications(predicted_labels, true_labels, raw_datasets['test'])

In [22]:
data = {'Predicted_Labels': predicted_labels}
df = pd.DataFrame(data)
df.to_csv('predicted_labels.csv', index=False)

Use the saved model

In [23]:
# Define the path to the saved model directory
saved_model_dir = "saved_model"

# Load the saved tokenizer
tokenizer = BertTokenizer.from_pretrained(saved_model_dir)

# Load the saved model
model = BertForSequenceClassification.from_pretrained(saved_model_dir)
model

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DistilBertTokenizer'. 
The class this function is called from is 'BertTokenizer'.


BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [24]:
trainer = Trainer(model=model)
predictions = trainer.predict(tokenized_datasets["test"])
# Apply softmax to convert logits to probabilities
probabilities = softmax(predictions.predictions, axis=1)

# Get the predicted class labels
predicted_labels = np.argmax(probabilities, axis=1)

print("Probabilities:\n", probabilities)
print("Predicted Labels:\n", predicted_labels)

Probabilities:
 [[0.6177605  0.38223955]
 [0.62697375 0.37302625]
 [0.6061025  0.3938975 ]
 [0.6224427  0.37755725]
 [0.63858867 0.36141133]
 [0.6199468  0.38005325]
 [0.6071735  0.39282653]
 [0.60732454 0.39267552]
 [0.6354282  0.36457175]
 [0.62641764 0.3735823 ]]
Predicted Labels:
 [0 0 0 0 0 0 0 0 0 0]
